In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [2]:
from cross_signal import *
from config_paper import paper_key_id, paper_secret_key

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key ,base_url)

In [ ]:
#import api2 for real trading
from config_paper import key_id, secret_key
api = tradeapi.REST(key_id, secret_key)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-01'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.cash

ACTIVE


'14867.54'

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [ ]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    
    #asset Manager
    assetlist=api.list_positions()
    
    for asset in assetlist:
            #if asset is in list of positions AND is more than 2.4% loss
            # sell and turn that symbol to null
        if float(asset.unrealized_intraday_plpc)< -.024 :
            openPosition = api.get_position(asset.symbol)

            returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day")
            assetsToTrade[i]= asset.symbol+' nulled for below -2.4% pnl ' +str(currenttime)
            print('Sell', asset.symbol)    
             
            
            
    #buy and sell        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df
            data=populateindicators(returned_data)
        except:
            print(symbol)
        
        
        
#buy conditions -------
        try:
            #set last time to buy stocks and when to start buying
            if currenttime <=1110 and currenttime >=636:    
            
            
            # Calculated trading indicators
            #populate exponential moving averages




                 # Create the buy signal here
                if data['adx'][-1] > 22 and \
                data['mfi'][-1] < 25 and \
                data['ultosc'][-1] < 25 and \
                data['rsi'][-1] < 33 and \
                data['minusdi'][-1] > 25 and \
                data['minusdi'][-1] > data['plusdi'][-1]:

                    buyord=True
                    assetlist=api.list_positions()
                    for asset in assetlist:
                        # if asset is in the list of positions we have already bought, then do not buy again
                        if asset.symbol == symbol:
                            buyord=False
                            print(asset.symbol,asset.qty)


                    




                    #if asset is not already bought, then buy             
                    if buyord==True:
                        #cashBalance = account.daytrading_buying_power
                        #equity
                        cashBalance = float(account.cash)

                        targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                        returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                        print('Buy ',returned)
        except:
            print(symbol)
                
         
        
        
#Sell conditions ------
        try:
            if data['rsi'][-1] > 71:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","limit","day",data['close'][-1]) # Market order to fully close position
                print('Sell',returned)
        except:
            print(symbol)

        
    
    
    #get current time and shut off trading after 12
     
    if currenthour >= 12: 
        trade=False
        print('Good work, Turning off')
    

    
    

BRKB
BFB
DVA
EXR
MKTK
RTN
TPR
UTX

BRKB
BFB
MKTK
RTN
TPR
UTX

BRKB
BFB
EXR
MKTK
PSA
RTN
TPR
UTX

BRKB
BFB
EXR
MKTK
PSA
RTN
UTX

BRKB
BFB
EIX
EXR
MKTK
PM
PSA
RTN
TPR
UTX

BRKB
BFB
EIX
EXR
MKTK
PSA
RTN
TPR
UTX

BRKB
BFB
EIX
EXR
MKTK
PSA
RTN
TPR
UTX

BRKB
BFB
EXR
MKTK
PSA
RTN
UTX

AGN
BRKB
BFB
EXR
MKTK
PSA
RTN
UTX

BRKB
BFB
EXR
MKTK
PSA
RTN
UTX

BRKB
BFB
EXR
MKTK
NOC
PSA
RTN
UTX

AGN
BRKB
BFB
EXR
GPS
MKTK
RTN
ROST
UTX

AGN
ADS
BRKB
BFB
CCL
EXR
GPS
MKTK
RTN
ROST
UTX

AGN
ADS
BRKB
BFB
CCL
EXR
MKTK
RTN
UTX

AGN
ADS
BRKB
BFB
CVX
EXR
IP
MKTK
RTN
UTX
WDC

AGN
ADS
BRKB
BFB
CVX
Sell Order({   'asset_class': 'us_equity',
    'asset_id': 'c997b367-2a67-472c-910f-d0324c599cea',
    'canceled_at': None,
    'client_order_id': '09ba267b-71e7-4a12-bbca-7c3b04106cc6',
    'created_at': '2020-05-01T17:07:48.253328Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '61fa0052-0f2d-47ea-8aa7-461

BRKB
BFB
MKTK
MA
RTN
UTX
HOG nulled for below -2.4% pnl 1024
BRKB
BFB
MKTK
RTN
UTX
HOG nulled for below -2.4% pnl 1024
BRKB
BFB
MKTK
RTN
UTX
HOG nulled for below -2.4% pnl 1024
BRKB
BFB
